In [1]:
import time

import numpy as np

import tensorflow as tf
import tensorflow.keras.backend as K

from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, AveragePooling2D
from tensorflow.keras.layers import Softmax, Reshape
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.losses import Huber
from tensorflow.keras.metrics import AUC, BinaryAccuracy, Accuracy, Recall, Precision
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, LearningRateScheduler, ReduceLROnPlateau, Callback
from keras.preprocessing.image import ImageDataGenerator

from HWlayer_base import evaluate_build, focus_build
from HWlayer_keras import HWlayer

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train / 255
y_train = tf.keras.utils.to_categorical(y_train, 10)

x_test = x_test / 255
y_test = tf.keras.utils.to_categorical(y_test, 10)

In [3]:
evaluate_num = 8
focus = 0.8
deepth = 4

evaluate_dims = evaluate_num * x_test.shape[-1]

In [4]:
def evaluate_build_AveragePooling2D(data, evaluate_num=8, focus=0.8, deepth=3):
    output_list = []
    x = Input(data.shape[1:])
    y_list = [x]
    for idx in range(deepth):
        y_list += [AveragePooling2D(2)(y_list[-1])]
    
    model = Model(x, y_list)
    data_list = model.predict(data)
    
    output_list = []
    for data in data_list:
        print(data.shape)
        evaluate_list = [evaluate_build(data[..., i], evaluate_num) for i in range(data.shape[-1])]
        evaluate_focus_list = [focus_build(evaluate, focus) for evaluate in evaluate_list]
        output_list.append(evaluate_focus_list)
        
    return output_list

evaluate_focus_list = evaluate_build_AveragePooling2D(x_test, evaluate_num, focus, deepth)

(10000, 32, 32, 3)


evaluate_num:   8,focus:0.8000: 8it [00:00, ?it/s]
evaluate_num:   8,focus:0.8000: 8it [00:00, ?it/s]
evaluate_num:   8,focus:0.8000: 8it [00:00, ?it/s]


(10000, 16, 16, 3)


evaluate_num:   8,focus:0.8000: 8it [00:00, ?it/s]
evaluate_num:   8,focus:0.8000: 8it [00:00, ?it/s]
evaluate_num:   8,focus:0.8000: 8it [00:00, 8031.22it/s]


(10000, 8, 8, 3)


evaluate_num:   8,focus:0.8000: 8it [00:00, ?it/s]
evaluate_num:   8,focus:0.8000: 8it [00:00, ?it/s]
evaluate_num:   8,focus:0.8000: 8it [00:00, 8017.79it/s]


(10000, 4, 4, 3)


evaluate_num:   8,focus:0.8000: 8it [00:00, ?it/s]
evaluate_num:   8,focus:0.8000: 8it [00:00, 8019.70it/s]
evaluate_num:   8,focus:0.8000: 8it [00:00, ?it/s]


(10000, 2, 2, 3)


evaluate_num:   8,focus:0.8000: 8it [00:00, 7972.07it/s]
evaluate_num:   8,focus:0.8000: 8it [00:00, 8019.70it/s]
evaluate_num:   8,focus:0.8000: 8it [00:00, ?it/s]


In [5]:
# VGG5_cfg = [64, 'M', 128, 'M', 256, 'M', 512, 'M', 512, 'M']
# VGG5_cfg = [32, 'A', 64, 'A', 128, 'A', 256, 'A', 256, 'A']
# VGG16_cfg = [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M']

VGG5_cfg = [16, 'M', 32, 'M', 64, 'M', 128, 'M', 128, 'M']
VGG16_cfg = [8, 8, 'A', 16, 16, 'A', 32, 32, 32, 'A', 64, 64, 64, 'A', 64, 64, 64, 'A']

In [6]:
x = Input(shape=((32, 32, 3)))
y = x
hw_active_dic = {}
for evaluate_focus in evaluate_focus_list:
    image_size = y.shape[-2]
    l = HWlayer(evaluate_focus, name=f'HW_active_{image_size}')(y)
    l = K.expand_dims(l, -2)
    hw_active_dic[image_size] = l
    y = MaxPooling2D(2)(y)

y = HWlayer(evaluate_focus_list[0])(x)
for cfg in VGG5_cfg:
    if cfg == 'M':
        y = MaxPooling2D(2, 2)(y)
    elif cfg == 'A':
        y = AveragePooling2D(2, 2)(y)
    else:
        y = Conv2D(cfg*evaluate_dims, 3, 1, 'same')(y)
        y = BatchNormalization()(y)
        y = Activation('relu')(y)
        
        output_shape = y.shape[1:]
        inter_shape = output_shape[:-1] + [cfg, evaluate_dims]
        y = Reshape(inter_shape)(y)
        # 这里可以增加一个全链接层
        # y = Dense(evaluate_dims)(y)
        # y = BatchNormalization()(y)
        # y = Activation('relu')(y)
        # 稀疏激活
        image_size = inter_shape[1]
        y = y * hw_active_dic[image_size]
        y = Reshape(output_shape)(y)

y = Flatten()(y)
y = Dense(10)(y)
y = Softmax(axis=-1)(y)

model = Model(x, y)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
h_wlayer (HWlayer)              (None, 32, 32, 24)   0           input_2[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 384)  83328       h_wlayer[0][0]                   
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32, 32, 384)  1536        conv2d[0][0]                     
____________________________________________________________________________________________

In [7]:
datagen = ImageDataGenerator(
    featurewise_center = False,  # 将整个数据集的均值设为 0
    samplewise_center = False,  # 将每个样本的均值设为 0
    featurewise_std_normalization = False,  # 将输入除以整个数据集的标准差
    samplewise_std_normalization = False,  # 将输入除以其标准差
    zca_whitening = False,  # 运用 ZCA 白化
    zca_epsilon = 1e-06,  # ZCA 白化的 epsilon值
    rotation_range = 0,  # 随机旋转图像范围 (角度, 0 to 180)
    width_shift_range = 0.1,  # 随机水平移动图像 (总宽度的百分比) 
    height_shift_range = 0.1,  # 随机垂直移动图像 (总高度的百分比)
    shear_range = 0.,  # 设置随机裁剪范围
    zoom_range = 0.,  # 设置随机放大范围
    channel_shift_range = 0.,  # 设置随机通道切换的范围
    fill_mode = 'nearest',  # 设置填充输入边界之外的点的模式
    cval = 0.,  # 在 fill_mode = "constant" 时使用的值
    horizontal_flip = True,  # 随机水平翻转图像
    vertical_flip = True,  # 随机垂直翻转图像
    rescale = None,  # 设置缩放因子 (在其他转换之前使用)
    preprocessing_function = None,  # 设置将应用于每一个输入的函数
    data_format = None,  # 图像数据格式，"channels_first" 或 "channels_last" 之一
    validation_split = 0.0)  # 保留用于验证的图像比例（严格在 0 和 1之间）

datagen.fit(x_train)

In [8]:
class LossHistory(Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.lr = []

    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs['loss'])
        self.lr.append(step_decay(len(self.losses)))

# lr_base = 0.001
# epochs = 250
# lr_power = 0.9
# def lr_scheduler(epoch, mode='power_decay'):
#     '''if lr_dict.has_key(epoch):
#         lr = lr_dict[epoch]
#         print 'lr: %f' % lr'''
 
#     if mode is 'power_decay':
#         # original lr scheduler
#         lr = lr_base * ((1 - float(epoch) / epochs) ** lr_power)
#     if mode is 'exp_decay':
#         # exponential decay
#         lr = (float(lr_base) ** float(lr_power)) ** float(epoch + 1)
#     # adam default lr
#     if mode is 'adam':
#         lr = 0.001
 
#     if mode is 'progressive_drops':
#         # drops as progression proceeds, good for sgd
#         if epoch > 0.9 * epochs:
#             lr = 0.0001
#         elif epoch > 0.75 * epochs:
#             lr = 0.001
#         elif epoch > 0.5 * epochs:
#             lr = 0.01
#         else:
#             lr = 0.1
 
#     print('lr: %f' % lr)
#     return lr

# callback_list = [LossHistory(), LearningRateScheduler(lr_scheduler),]

In [9]:
lr_base = 0.01
epochs = 250
lr_power = 0.9

def scheduler(epoch, lr):
    lr = lr_base * ((1 - float(epoch) / epochs) ** lr_power)
    return lr

In [ ]:
model_name = time.strftime('CIFAR10_VGG_KERAS_HWNET%Y%m%d%H%M%S')

model.compile(loss=Huber(), optimizer=RMSprop(learning_rate=0.0001, decay=1e-6), metrics=[ 'acc', 'AUC'])

callback_list = [
    EarlyStopping(monitor='loss', patience=32),
    ModelCheckpoint('models/%s_best.h5'%model_name, monitor='loss', save_best_only=True, verbose=False),
    TensorBoard(log_dir='./Log/%s'%model_name)
]

model.fit(datagen.flow(x_train, y_train, batch_size=100), epochs=epochs, validation_data=(x_test, y_test), callbacks=callback_list)

Epoch 1/250
469/500 [===========================>..] - ETA: 8s - loss: 0.0349 - acc: 0.4334 - auc: 0.8519